In [8]:
#Random Forest (Binar)
from sklearn.ensemble import RandomForestClassifier

In [11]:
rf_model = RandomForestClassifier(
    n_estimators=300,
    random_state=42
)


In [12]:
rf_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("model", rf_model)
])


In [13]:
gkf = GroupKFold(n_splits=5)

acc_scores = []
sens_scores = []
spec_scores = []
auc_scores = []

for train_idx, test_idx in gkf.split(X, y_bin, groups=pat_id):
    X_tr, X_te = X.iloc[train_idx], X.iloc[test_idx]
    y_tr, y_te = y_bin.iloc[train_idx], y_bin.iloc[test_idx]

    rf_pipe.fit(X_tr, y_tr)                 # TRAIN
    prob = rf_pipe.predict_proba(X_te)[:,1] # probability of inflammation
    pred = (prob >= 0.5).astype(int)        # convert to 0/1


In [14]:
    acc_scores.append(accuracy_score(y_te, pred))

    tn, fp, fn, tp = confusion_matrix(y_te, pred).ravel()

    sens = tp / (tp + fn) if (tp + fn) else np.nan
    sens_scores.append(sens)

    spec = tn / (tn + fp) if (tn + fp) else np.nan
    spec_scores.append(spec)

    auc_scores.append(roc_auc_score(y_te, prob))


In [15]:
print("Random Forest (Binary <3 vs ≥3) with GroupKFold")
print(f"Accuracy:    {np.mean(acc_scores):.3f} ± {np.std(acc_scores):.3f}")
print(f"Sensitivity: {np.mean(sens_scores):.3f} ± {np.std(sens_scores):.3f}")
print(f"Specificity: {np.mean(spec_scores):.3f} ± {np.std(spec_scores):.3f}")
print(f"ROC-AUC:     {np.mean(auc_scores):.3f} ± {np.std(auc_scores):.3f}")


Random Forest (Binary <3 vs ≥3) with GroupKFold
Accuracy:    0.952 ± 0.000
Sensitivity: 0.941 ± 0.000
Specificity: 0.964 ± 0.000
ROC-AUC:     0.982 ± 0.000
